In [ ]:
from ipyleaflet import Map, basemaps, Marker, WidgetControl

from ipywidgets import IntSlider, ColorPicker, Button, link
import ipywidgets as widgets

from ipyfilechooser import FileChooser

from shapely import Point

from pathlib import Path
import geopandas as gp
import numpy as np
import xarray as xr
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import pandas as pd

from pypism.utils import tqdm_joblib
from pypism.pathlines import compute_pathline, row_to_pathline_geopandas_dataframe


In [ ]:
center = [72, -45]
zoom = 2.75
crs = "EPSG:3413"

In [ ]:
raster_file_widget = widgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)

In [ ]:
raster_file_upload_widget= widgets.Button(
    description='Submit',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)
display(raster_file_upload_widget)

In [ ]:
marker = Marker(location=center, draggable=True)

In [ ]:
m = Map(basemap=basemaps.Esri.WorldImagery, center=center, zoom=zoom)
m.add(marker)

In [ ]:
def on_location_changed(event):
    # Do some computation given the new marker location, accessible from `event['new']`
    pass

marker.observe(on_location_changed, 'location')

In [ ]:
starting_point = gp.GeoSeries(Point(marker.location), crs="EPSG:4326").to_crs(crs)

In [ ]:
main_display = widgets.Output()

def show_it(inputs):
    with main_display:
        main_display.clear_output()
        display(list(inputs['new'].keys())[-1])

upload = widgets.FileUpload(
    accept='.nc',
    multiple=False
)

upload.observe(show_it, names='value')

In [ ]:
display(upload)

In [ ]:
# Create and display a FileChooser widget
fc = FileChooser('.')
display(fc)


In [ ]:
raster_url = fc.selected

In [ ]:
ds = xr.open_dataset(raster_url)

In [ ]:
vx_select = widgets.Select(
    options=list(ds.data_vars),
    value="vx" if "vx" in ds.data_vars else None,
    # rows=10,
    description='velocity x-component:',
    disabled=False
)
vy_select = widgets.Select(
    options=list(ds.data_vars),
    value="vy" if "vy" in ds.data_vars else None,
    # rows=10,
    description='velocity y-component:',
    disabled=False
)

In [ ]:
display(vx_select)
display(vy_select)

In [ ]:
vx_varname = vx_select.value
vy_varname = vy_select.value

In [ ]:
Vx = np.squeeze(ds[vx_varname].to_numpy())
Vy = np.squeeze(ds[vy_varname].to_numpy())
x = ds["x"].to_numpy()
y = ds["y"].to_numpy()


In [ ]:
dt = 1
total_time = 1_000
reverse = True

In [ ]:
compute_pathline(
                [*starting_point.geometry[0].coords[0]],
                Vx,
                Vy,
                x,
                y,
                dt=dt,
                total_time=total_time,
                reverse=reverse,
                progress=True,
                progress_kwargs= {"leave": False, "position": 1},
)
